In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from numpy import ndarray
import SimpleITK
from matplotlib import pyplot as plt

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper parameters
num_epochs = 200
learning_rate = 0.0005
batch = 48 

In [4]:
class Dataset(Dataset):
    def __init__(self, data, target, transform = None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).float()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
# Load train data

train_dataset = Dataset(np.load('forcetrain.npy'), np.load('disptrain.npy'))
train_loader = DataLoader(
    train_dataset,
    batch_size = batch,
    shuffle = True,
    num_workers = 2,
    pin_memory = True 
)

for batch_idx, (data, target) in enumerate(loader):
    print('Train Batch idx {}, force shape {}, displacement shape {}'.format(
        batch_idx, data.shape, target.shape))

In [ ]:
# Load test data

test_dataset = Dataset(np.load('forcetest.npy'), np.load('disptest.npy'))
test_loader = DataLoader(
    test_dataset,
    batch_size = batch,
    shuffle = True,
    num_workers = 2,
    pin_memory = True 
)

for batch_idx, (data, target) in enumerate(test_loader):
    print('Test Batch idx {}, force shape {}, displacement shape {}'.format(
        batch_idx, data.shape, target.shape))

In [ ]:
# Define the main 3D UNET blocks

def conv_block_3d(in_dim, out_dim, activation):
    return nn.Sequential(
        nn.Conv3d(in_dim, out_dim, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm3d(out_dim),
        activation,)


def conv_trans_block_3d(in_dim, out_dim, activation):
    return nn.Sequential(
        nn.ConvTranspose3d(in_dim, out_dim, kernel_size=3, stride=2, padding=1, output_padding=1),
        nn.BatchNorm3d(out_dim),
        activation,)


def max_pooling_3d():
    return nn.MaxPool3d(kernel_size=2, stride=2, padding=0)


def conv_block_2_3d(in_dim, out_dim, activation):
    return nn.Sequential(
        conv_block_3d(in_dim, out_dim, activation),
        nn.Conv3d(out_dim, out_dim, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm3d(out_dim),
        activation,)

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_dim, out_dim, num_filters):
        super(UNet, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.num_filters = num_filters
        activation = nn.LeakyReLU(0.2, inplace=True)
        
        # Down sampling
        self.down_1 = conv_block_2_3d(self.in_dim, self.num_filters, activation)
        self.pool_1 = max_pooling_3d()
        self.down_2 = conv_block_2_3d(self.num_filters, self.num_filters * 2, activation)
        self.pool_2 = max_pooling_3d()
        self.down_3 = conv_block_2_3d(self.num_filters * 2, self.num_filters * 4, activation)
        self.pool_3 = max_pooling_3d()
        self.down_4 = conv_block_2_3d(self.num_filters * 4, self.num_filters * 8, activation)
        self.pool_4 = max_pooling_3d()
        self.down_5 = conv_block_2_3d(self.num_filters * 8, self.num_filters * 16, activation)
        self.pool_5 = max_pooling_3d()
        
        # Bridge
        self.bridge = conv_block_2_3d(self.num_filters * 16, self.num_filters * 32, activation)
        
        # Up sampling
        self.trans_1 = conv_trans_block_3d(self.num_filters * 32, self.num_filters * 32, activation)
        self.up_1 = conv_block_2_3d(self.num_filters * 48, self.num_filters * 16, activation)
        self.trans_2 = conv_trans_block_3d(self.num_filters * 16, self.num_filters * 16, activation)
        self.up_2 = conv_block_2_3d(self.num_filters * 24, self.num_filters * 8, activation)
        self.trans_3 = conv_trans_block_3d(self.num_filters * 8, self.num_filters * 8, activation)
        self.up_3 = conv_block_2_3d(self.num_filters * 12, self.num_filters * 4, activation)
        self.trans_4 = conv_trans_block_3d(self.num_filters * 4, self.num_filters * 4, activation)
        self.up_4 = conv_block_2_3d(self.num_filters * 6, self.num_filters * 2, activation)
        self.trans_5 = conv_trans_block_3d(self.num_filters * 2, self.num_filters * 2, activation)
        self.up_5 = conv_block_2_3d(self.num_filters * 3, self.num_filters * 1, activation)
        
        # Output
        self.out = conv_block_3d(self.num_filters, out_dim, activation)
    
    def forward(self, x):
        # Down sampling
        down_1 = self.down_1(x) 
        pool_1 = self.pool_1(down_1) 
        
        down_2 = self.down_2(pool_1) 
        pool_2 = self.pool_2(down_2) 
        
        down_3 = self.down_3(pool_2) 
        pool_3 = self.pool_3(down_3) 
        
        down_4 = self.down_4(pool_3) 
        pool_4 = self.pool_4(down_4) 
        
        down_5 = self.down_5(pool_4) 
        pool_5 = self.pool_5(down_5) 
        
        # Skip Connection
        bridge = self.bridge(pool_5) 
        
        # Upsampling
        trans_1 = self.trans_1(bridge) 
        concat_1 = torch.cat([trans_1, down_5], dim=1) 
        up_1 = self.up_1(concat_1) 
        
        trans_2 = self.trans_2(up_1) 
        concat_2 = torch.cat([trans_2, down_4], dim=1) 
        up_2 = self.up_2(concat_2) 
        
        trans_3 = self.trans_3(up_2) 
        concat_3 = torch.cat([trans_3, down_3], dim=1) 
        up_3 = self.up_3(concat_3) 
        
        trans_4 = self.trans_4(up_3) 
        concat_4 = torch.cat([trans_4, down_2], dim=1) 
        up_4 = self.up_4(concat_4) 
        
        trans_5 = self.trans_5(up_4) 
        concat_5 = torch.cat([trans_5, down_1], dim=1) 
        up_5 = self.up_5(concat_5) 
        
        # Output
        out = self.out(up_5)
        return out

In [ ]:
model = UNet(in_dim = 3, out_dim = 3, num_filters = 4).to(device)

# Loss and optimizer
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25,50,75], gamma=0.1)

In [ ]:
# Train the model
total_step = len(train_loader)
loss_values = []
loss_values_test = []

for epoch in range(num_epochs):
    
    running_loss = 0.0
    running_loss_test = 0.0

    
    for i, (data, target) in enumerate(train_loader):
        forcet = data.to(device)
        dispt = target.to(device)
        
        # Forward pass
        outputs = model(forcet)
        loss = criterion(outputs, dispt)
        running_loss =+ loss.item() * forcet.size(0)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if (i+1) % 2 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
         
     
    loss_values.append(running_loss / len(train_loader))
     
    
    for i, (data, target) in enumerate(test_loader):
        
        model.eval()
        with torch.no_grad():
            
            forceTest = data.to(device)
            dispTest = target.to(device)
            outputs_test = model(forceTest)
            loss = criterion(outputs_test, dispTest)
            running_loss_test =+ loss.item() * forceTest.size(0)
        
    
    loss_values_test.append(running_loss_test / len(train_loader)) 
    #scheduler.step()
    

fig, ax = plt.subplots()
ax.plot(loss_values, label = 'Training Loss')
ax.plot(loss_values_test, label = 'Validation Loss')
ax.set(xlabel='No. of Epochs', ylabel ='Loss',
       title='Training Results')
ax.grid()
plt.legend()
#fig.savefig("200ep-0.0001lr-48batchsize.png")
plt.show()

In [ ]:
# Training output
out_train = outputs.detach().cpu().numpy()
out1_train = out_train[-1]
out2_train = np.swapaxes(out1_train, 0, 3)
out2_train.shape

# Training output plot
fig = plt.figure()
plt.imshow(out2_train[:,:,35,2])
plt.show()

In [ ]:
# Training groundtruth
true_train = dispt.detach().cpu().numpy()
true1_train = true_train[-1]
true2_train = np.swapaxes(true1_train, 0, 3)
true2_train.shape

# Training groundtruth plot
fig = plt.figure()
plt.imshow(true2_train[:,:,35,2])
plt.show()

In [ ]:
# Validation output
out_valid = outputs_test.cpu().numpy()
out1_valid = out_valid[-1]
out2_valid = np.swapaxes(out1_valid, 0, 3)
out2_valid.shape

# Validation output plot
fig = plt.figure()
plt.imshow(out2_valid[:,:,35,2])
plt.show()

In [ ]:
# Validation set groundtruth
true_valid = dispTest.cpu().numpy()
true1_valid = true_valid[-1]
true2_valid = np.swapaxes(true1_valid, 0, 3)
true2_valid.shape

# Validation set groundtruth plot
fig = plt.figure()
plt.imshow(true2_valid[:,:,35,2])
plt.show()

In [ ]:
train_input = forcet.detach().cpu().numpy()
test_input = forceTest.cpu().numpy()

In [ ]:
trainip = np.swapaxes(train_input[-1], 0, 3)
testip = np.swapaxes(test_input[-1], 0, 3)

In [ ]:
print(np.shape(true_train))
print(np.shape(out_train))
print(np.shape(true_valid))
print(np.shape(out_valid))
print(np.shape(train_input))
print(np.shape(test_input))

In [ ]:
np.save('truetrain.npy', true_train)
np.save('outtrain.npy', out_train)
np.save('truevalid.npy', true_valid)
np.save('outvalid.npy', out_valid)
np.save('traininput.npy', train_input)
np.save('testinput.npy', test_input)

In [ ]:
torch.save(model.state_dict(), '/home/oyoussef/200epochs-48batch-0.0005lr-SmoothL1Loss.pth')

In [ ]:
def write_raw(data: ndarray, filename):
    """
    Writes data to a .raw and .mhd file.

    The provided data will be written to the filesystem as a .mhd file
    containing the information and a .raw containing the actual data.

    Args:
        data (ndarray): The data which will be printed in the file.
        filename: The name of the file. It has to end with .mhd.
    """

    if not filename.endswith(".mhd"):
        raise FileExtensionError("Provided file has to be of type .mhd")

    image = SimpleITK.GetImageFromArray(data)
    f = open(filename, "w")
    SimpleITK.WriteImage(image, filename)
    f.close()



In [ ]:
# Writing the data as metaimages for visualization in Paraview

write_raw(out2_train, 'Training_Output.mhd')
write_raw(true2_train, 'Training_GT.mhd')

write_raw(out2_valid, 'Validation_Output.mhd')
write_raw(true2_valid, 'Validation_GT.mhd')

write_raw(trainip, 'Training_Input.mhd')
write_raw(testip, 'Validation_Input.mhd')